In [2]:
print("Hello, Jupyter!")

Hello, Jupyter!


In [1]:
import tensorflow as tf
from tensorflow.keras.models import load_model
import pickle
import pandas as pd
import numpy as np

In [3]:
### Loath the trained model, scalarpickle,onehot
model=load_model('model.h5')

## Load the encoder and scaler
with open('onehot_encoder_geo.pkl', 'rb') as file:
    label_encoder_geo=pickle.load(file)

with open('label_encoder_gender.pkl', 'rb') as file:
    label_encoder_gender=pickle.load(file)
  
with open('scaler.pkl', 'rb') as file:
    scaler=pickle.load(file)

In [4]:
### Example input data
input_data={
    'CreditScore':600, 
    'Geography':'France',
    'Gender':'Male',
    'Age':40,
    'Tenure':3,
    'Balance':60000,
    'NumOfProducts':2,
    'HasCrCard':1,
    'IsActiveMember':1,
    'EstimatedSalary':50000
}

In [5]:
## One hot encode Geography
geo_encoded=label_encoder_geo.transform([[input_data['Geography']]]).toarray()
geo_encoded_df=pd.DataFrame(geo_encoded, columns=label_encoder_geo.get_feature_names_out(['Geography']))
print(geo_encoded_df)

   Geography_France  Geography_Germany  Geography_Spain
0               1.0                0.0              0.0


/Users/pandeyraj/miniforge3/envs/tf310/lib/python3.10/site-packages/sklearn/utils/validation.py:2749: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  warnings.warn(


In [6]:
input_df=pd.DataFrame([input_data])
input_df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,600,France,Male,40,3,60000,2,1,1,50000


In [7]:
### Combine onehot encoded columns with input data
input_df['Gender']=label_encoder_gender.transform(input_df['Gender'])
input_df                                                               
                                                 

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,600,France,1,40,3,60000,2,1,1,50000


In [8]:
## Concatination with one hot encoded 
input_df=pd.concat([input_df.drop('Geography', axis=1),geo_encoded_df], axis=1)
print(input_df)

   CreditScore  Gender  Age  Tenure  Balance  NumOfProducts  HasCrCard  \
0          600       1   40       3    60000              2          1   

   IsActiveMember  EstimatedSalary  Geography_France  Geography_Germany  \
0               1            50000               1.0                0.0   

   Geography_Spain  
0              0.0  


In [9]:
## Scaling the input data
input_scaled=scaler.transform(input_df)
print(input_scaled)

[[-0.53598516  0.91324755  0.10479359 -0.69539349 -0.25781119  0.80843615
   0.64920267  0.97481699 -0.87683221  1.00150113 -0.57946723 -0.57638802]]


In [10]:
## Predict churn
prediction=model.predict(input_scaled)
print("Prediction:", prediction)

1/1 [==============================] - 0s 77ms/step
Prediction: [[0.06152005]]


2026-02-10 20:16:09.369992: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


In [12]:
prediction_proba=prediction

In [13]:
prediction_proba

array([[0.06152005]], dtype=float32)

In [14]:
if prediction_proba[0][0]>0.5:
    print("The customer is likely to churn.")
else:
    print("The customer is unlikely to churn.")

The customer is unlikely to churn.
